In [25]:
from jaco.models.wind_comparison import cooling, heating
from sympy.utilities.codegen import C99CodeGen, make_routine
from sympy.codegen.ast import Assignment
import sympy as sp
system = cooling + heating
solve_vars = ['u','T']
time_dependent = ['T']
func, jac, indices = system.solver_functions(solve_vars, time_dependent, return_jac=True)
funcjac = sp.Matrix(func+sp.flatten(jac))
X = sp.MatrixSymbol("X",len(indices),1)
for var in indices:
    funcjac = funcjac.subs(var,X[indices[var]]) #.subs("T",sp.MatrixSymbol("X",2,1)[1]).subs("u",sp.MatrixSymbol("X",2,1)[0])
paramsvars=funcjac.free_symbols.copy()
paramsvars.remove(X)
P = sp.MatrixSymbol('params',len(paramsvars),1)
assignments = []
for i, p in enumerate(paramsvars):
    funcjac = funcjac.subs(p, P[i])
    assignments.append(Assignment(P[i],p))
cg = C99CodeGen(cse=True)
routine = cg.routine("microphysics_func_jac",funcjac) #cg.routine("func",sp.Matrix(func + sp.flatten(jac)))
cg.write([routine],"microphysics_func_jac",to_files=True)



In [24]:
sp.ccode(assignments)

'{params[0] = n_Htot;, params[1] = u_0;, params[2] = Δt;, params[3] = x_H;}'

In [3]:
from sympy.utilities.codegen import FCodeGen
import sympy as sp
cg = FCodeGen()
routine =cg.routine("func_jac",funcjac) #cg.routine("func",sp.Matrix(func + sp.flatten(jac)))
#C99CodeGen(cse=True).dump_c([routine], open("cooling.c","w"),"prefix")
cg.write([routine],"func_jac",to_files=True)

In [4]:
sp.flatten(jac)

[-1,
 122700210.820621,
 0,
 -n_Htot**2*x_H**2*Piecewise((0, (log(T) >= 27.6310211159285) | (log(T) < 2.30258508299405)), (2.99999998697116/T, log(T) < 4.60517018598809), (0.731675663351629/T, log(T) < 8.69951474821019), (8.33549431289496/T, log(T) < 9.76995615991161), (-0.269927827516604/T, log(T) < 10.5966347330961), (1.89942351828434/T, log(T) < 11.3736633976367), (-0.00984338058146309/T, log(T) < 12.3458345879053), (-1.86982630031235/T, log(T) < 12.7938593104323), (-0.41187017623273/T, log(T) < 14.2209756660724), (-1.50238273377814/T, log(T) < 15.0682735264596), (-0.254733492486591/T, log(T) < 17.0736070959858), (0.500035898435738/T, True))*exp(Piecewise((-69.0775527898214, log(T) < 2.30258508299405), (-46.9384337895135, log(T) >= 27.6310211159285), (2.99999998697116*log(T) - 75.9853080088035, log(T) < 4.60517018598809), (0.731675663351629*log(T) - 65.5392884615192, log(T) < 8.69951474821019), (8.33549431289496*log(T) - 131.688820945937, log(T) < 9.76995615991161), (-0.269927827516

In [5]:
funcjac

Matrix([
[                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

In [6]:
sp.Matrix(jac).subs("T",sp.MatrixSymbol("X",2,1)[1])

Matrix([
[-1,                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

In [7]:
from sympy.vector import VectorSymbol
Vector("X")

ImportError: cannot import name 'VectorSymbol' from 'sympy.vector' (/Users/mgrudic/Library/Python/3.13/lib/python/site-packages/sympy/vector/__init__.py)